In [6]:
import os
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import pipeline

os.environ['http_proxy'] = "http://proxy61.iitd.ac.in:3128"
os.environ['https_proxy'] = "http://proxy61.iitd.ac.in:3128"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [9]:
MODEL_NAME = "/home/models/google-flan-t5-small"
MAX_OUTPUT_LENGTH = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Define a function for text generation
def generate_text(prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=max_length, truncation=True).to(device)
    output_ids = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

Using device: cuda


In [8]:
# Example usage
prompt = """
Answer the given question correctly based on the given context.

Context: 
史瑞克是一个孤独的怪物，当童话生物都被法克大人送到他的沼泽时，他十分惊讶。他和一只叫驴子的驴成为朋友，并一起前往杜洛城与法克谈判。法克须要和费欧娜公主结婚才能成为杜洛城的国王。法克答应如果他们能从一只巨大的火龙中救费欧娜出来就还回他宁静的沼泽。两人到达城堡后发现火龙原来是女性（小龙女），而且她爱上了驴子。 The New Yorkers, though not particularly organized, called their activities Dada, but they did not issue manifestos. They issued challenges to art and culture through publications such as The Blind Man, Rongwrong, and New York Dada in which they criticized the traditionalist basis for museum art. New York Dada lacked the disillusionment of European Dada and was instead driven by a sense of irony and humor. In his book Adventures in the arts: informal chapters on painters, vaudeville and poets Marsden Hartley included an essay on "The Importance of Being 'Dada'". في أواخر مارس 2013 فتحت القوات الحكومية جبهة جديدة في بلدة العتيبة في محاولة لتحويل جهود الحرب ومهاجمة قوات المتمردين في الغوطة الشرقية من الخلف.في 7 أبريل شنت القوات الحكومية هجوما شرق دمشق حيث ادعت وكالة الأنباء السورية (سانا) أن الجيش فرض حصارا على منطقة الغوطة الشرقية التي يسيطر عليها المتمردون. ذكر المتمردون أن هجمات الجيش المكثفة على المنطقة كانت مستمرة منذ منتصف مارس. أكد أحد قادة المتمردين أن المدخل الشمالي للغوطة كان محاصرا وذكر ناشط معارض أنه تم شن هجوم حكومي بقيادة دبابات على الغوطة الشرقية باتجاه الجنوب من اتجاه مطار دمشق الدولي.في 15 أبريل قتل ما لا يقل عن 20 شخصا في غارات جوية وقذائف صاروخية في شمال وشرق دمشق. يقال أن المقاتلين المتمردين زادت قوتهم في حيي القابون وبرزة للحد من خطر استخدام الأسلحة الكيميائية عليهم حيث من المرجح أن يوقعوا إصابات في صفوف الجيش السوري يقاتلون المتمردين في أماكن قريبة.في 18 أبريل اغتال مسلح علي علان رئيس العلاقات العامة في وزارة الشؤون الاجتماعية وعضو وكالة الإغاثة السورية في مطعم في منطقة المزة.في 20 أبريل كان الجيش السوري يضغط من أجل السيطرة الكاملة على بلدة جديدة الفضل جنوب غرب دمشق. قتل 69 شخصا في المعركة التي استمرت أربعة أيام ومعظمهم من المتمردين. كما أثر القتال على بلدة جديدة عرطوز ذات الأغلبية المسيحية.

Question: द न्यू यॉर्कर्स के द्वारा क्या जारी नहीं किया गया था?

Answer:
"""
generated_text = generate_text(prompt, max_length=MAX_OUTPUT_LENGTH)
print(generated_text.strip())

69


In [12]:
input_ids = tokenizer.encode(prompt)
len(input_ids)

614

In [1]:
from transformers import AutoTokenizer

def check_sequence_length(input_text, model_name):
    """
    Checks if the input text length exceeds the maximum sequence length of the specified model.

    Args:
        input_text (str): The input text to check.
        model_name (str): The name of the Hugging Face model (e.g., "bert-base-uncased").

    Returns:
        str: A message indicating whether the input text length is within the model's limits.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    max_length = tokenizer.model_max_length

    input_tokens = tokenizer.tokenize(input_text)
    input_length = len(input_tokens)

    if input_length <= max_length:
        return f"The input text length ({input_length}) is within the model's maximum sequence length ({max_length})."
    else:
        return f"Warning: The input text length ({input_length}) exceeds the model's maximum sequence length ({max_length}). Consider truncating or splitting the text."

# Example usage:
input_text = "This is an example input text that we want to check."
model_name = "google/flan-t5-small"
result = check_sequence_length(input_text, model_name)
print(result)


/home/amey/miniconda3/envs/socionet/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The input text length (12) is within the model's maximum sequence length (512).


In [16]:
input_texts = ["This is an example input text that we want to check."]*100
tokenizer = AutoTokenizer.from_pretrained(model_name)
input_tokens = tokenizer.encode(input_text, add_special_tokens=False)
input_length = len(input_tokens)
print(input_tokens
)
print(input_length)

[100, 19, 46, 677, 3785, 1499, 24, 62, 241, 12, 691, 5]
12


In [17]:
tokenizer.decode(input_tokens)

'This is an example input text that we want to check.'

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from accelerate import Accelerator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize the accelerator
accelerator = Accelerator()

# Specify the model
model_name = "/home/models/google-flan-t5-small"

# Load the tokenizer and the model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

# Use accelerator.prepare to make sure everything is on the right device
model = accelerator.prepare(model)


Using device: cuda


In [18]:
# Now you can use your model for inference!
text = ["translate English to French: Hello, world!"]*100
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

# Use the accelerator's device placement for the inputs
inputs = accelerator.prepare(inputs).to(accelerator.device)

with accelerator.autocast():
    outputs = model.generate(**inputs)

# Decode the generated text
for output in outputs:
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"The generated text is: {generated_text}")

/home/amey/miniconda3/envs/socionet/lib/python3.9/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
The generated text is: Hello, world!
T

### Run inference pipeline for FlanT5

In [1]:
from flant5 import FlanT5
flant5 = FlanT5(
    max_seq_len=10
)

/home/amey/miniconda3/envs/socionet/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = """
पीनियल ग्रंथि को मूलतः किसी बड़े अंग का "अवशेष" रूप माना जाता था। 1917 में यह पता चला था कि गाय के पीनियल के रस ने मेंढ़क की त्वचा को चमका दिया था। येल विश्वविद्यालय में त्वचा-विज्ञान के प्रोफ़ेसर आरोन बी. लर्नर और उनके साथियों ने इस उम्मीद में कि पीनियल रस चर्म रोगों को ठीक करने में सहायक हो सकता है, 1958 में मेलाटोनिन हार्मोन को अलग किया और उसका यह नाम रखा।  यद्यपि यह पदार्थ वांछित रूप से मददगार साबित नहीं हुआ, लेकिन उसकी खोज अनेक अन्य रहस्यों को सुलझाने में सहायक रही, जैसे कि चूहे की पीनियल हटाने से उसकी डिंब ग्रंथी क्यों बढ़ जाती है, चूहों को लगातार प्रकाश में रखने से उनकी पीनियल का वज़न क्यों घट जाता है और पीनियल को काट कर निकाल देने तथा लगातार प्रकाश का प्रभाव समान रूप से डिंब ग्रंथि में बढ़ोतरी क्यों करता है; इस जानकारी ने तत्कालीन नए विषय-क्षेत्र  (SCG) तक और वहां से पीनियल ग्रंथि तक जाते हैं। मानव शरीर में पीनियल ग्रंथि की गतिविधि (यां) स्पष्ट नहीं है; आम तौर पर इसे जैवचक्रीय अनुक्रम निद्रा विकार के उपचार के लिए दिया जाता है। Gates was born in Seattle, Washington, on October 28, 1955. He is the son of William H. Gates Sr. (b. 1925) and Mary Maxwell Gates (1929–1994). His ancestry includes English, German, Irish, and Scots-Irish. His father was a prominent lawyer, and his mother served on the board of directors for First Interstate BancSystem and the United Way. Gates' maternal grandfather was J. W. Maxwell, a national bank president. Gates has one older sister, Kristi (Kristianne), and a younger sister, Libby. He is the fourth of his name in his family, but is known as William Gates III or "Trey" because his father had the "II" suffix. The family lived in the Sand Point . The family encouraged competition; one visitor reported that "it didn't matter whether it was hearts or succeeded Gates as Microsoft's CEO. He maintained that position from 2000 until his resignation from the company in 2014. गैर-ह्रदय संबंधी कारण आमतौर पर अप्रत्यक्ष होते हैं और इसमें दवा का इस्तेमाल या दुरुपयोग; चयापचय या अंतःस्त्रावी मुद्दे, खासकर गलग्रंथि में; एक इलेक्ट्रोलाइट असंतुलन; तंत्रिका संबंधी कारक; स्वचालित प्रतिवर्त; परिस्थितिजनक कारक जैसे लंबे समय तक विश्राम; और स्वरोगप्रतिरोध शामिल होते हैं।है; इस जानकारी ने तत्कालीन नए विषय-क्षेत्र  (SCG) तक और वहां से पीनियल ग्रंथि तक जाते हैं। मानव शरीर में पीनियल ग्रंथि की गतिविधि (यां) स्पष्ट नहीं है; आम तौर पर इसे जैवचक्रीय अनुक्रम निद्रा विकार के उपचार के लिए दिया जाता है।
"""
prompts = [prompt]
